In [1]:
from PIL import ImageFont, Image, ImageDraw, ImageChops, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import json

def font2array(font_ttf, chars, size = 64, ratio = 0.5):
    blank = Image.new('L', (size*4, size*4), 255)
    font = ImageFont.truetype(font_ttf, int(size))
    # We need to make sure we scale down the fonts but preserve the vertical alignment
    min_ly = float('inf')
    max_hy = float('-inf')
    max_width = 0
    imgs = []

    for char in chars:
        # Draw character
        img = Image.new("L", (size*3, size*3), 255)
        draw = ImageDraw.Draw(img)
        w, h = font.getsize(char)
#         draw.text((size//2, -20), char, font=font)
        draw.text(((size*3-w)//2,(size*3-h)//2), char, font=font)
#         img.show()
        # Get bounding box
        diff = ImageChops.difference(img, blank)
        if diff.getbbox() == None:
            print(f'char:{char} of font:{font_ttf}')
            return
        lx, ly, hx, hy = diff.getbbox()
        min_ly = min(min_ly, ly)
        max_hy = max(max_hy, hy)
        max_width = max(max_width, hx - lx)
        imgs.append((lx, hx, img))

    #print('crop dims:', max_hy - min_ly, max_width)
    scale_factor = min(ratio * size / (max_hy - min_ly), ratio * size / max_width)
    data = []

    for lx, hx, img in imgs:
        img = img.crop((lx, min_ly, hx, max_hy))

        # Resize to smaller
        new_width = (hx-lx) * scale_factor
        new_height = (max_hy - min_ly) * scale_factor
        img = img.resize((int(new_width), int(new_height)), Image.ANTIALIAS)

        # Expand to square
        img_sq = Image.new('L', (size, size), 255)
        offset_x = (size - new_width)/2
        offset_y = (size - new_height)/2
        img_sq.paste(img, (int(offset_x), int(offset_y)))

        # Convert to numpy array
        matrix = np.array(img_sq.getdata()).reshape((size, size))
        matrix = 255 - matrix
        data.append(matrix)

    return np.array(data)

In [2]:
def plot_font(data, font_name):
    fig = plt.figure(figsize=(9, 6))
    for i in range(data.shape[0]):
      plt.subplot(9, 6, i+1)
      plt.axis('off')
      plt.imshow(255-data[i], cmap = 'gray')
      plt.savefig(f'plot_font/{font_name[:-4]}.png', bbox_inches='tight')
    plt.clf()
    
def plot_all_font_chicken(data, fonts_name):
    data = data.transpose(1,0,2,3)[0]
    for i, font in zip(data, fonts_name):
        img = Image.fromarray(255-i)
        img.save(f"plot_font_chicken/{font}_chicken.png","PNG")

In [3]:
fonts = os.listdir('All_fonts/')
chars = 'กขฃฅคฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ๐๑๒๓๔๕๖๗๘๙'
print(len(fonts), len(chars))

405 54


In [14]:
#checking for courrupt fonts. 
for font in fonts:
    test = font2array(font_ttf = 'All_Fonts/' + font, chars = 'ฃ', size = 64, ratio = 0.8)
    if test is None:
        continue
    img = Image.fromarray(test[0].astype(np.uint8))
    img.save(f'check_font/{font[:-4]}.PNG', 'PNG')

In [4]:
fonts = os.listdir('All_Fonts')
num_fonts = len(fonts)

chars = 'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ๐๑๒๓๔๕๖๗๘๙'
num_chars = len(list(chars))

# numpy array data
data = np.array([])


size = 64
report_interval = 20
i = 0

print(f'#font:{len(fonts)}\n#char:{num_chars}')
start = time.time()

for font in fonts:
    char_font = font2array(font_ttf = 'All_Fonts/' + font, chars = chars,size = 64, ratio = 0.9)
    data = np.append(data, char_font)
    if i % report_interval == 0:
        print(f'finish font:{i+1}th')
    i += 1

print(f'time use: {time.time()-start}')

print(data.shape)
data = data.astype(np.uint8)
data = data.reshape(-1, num_chars, size, size)
print(data.shape)

#font:405
#char:54
finish font:1th
finish font:21th
finish font:41th
finish font:61th
finish font:81th
finish font:101th
finish font:121th
finish font:141th
finish font:161th
finish font:181th
finish font:201th
finish font:221th
finish font:241th
finish font:261th
finish font:281th
finish font:301th
finish font:321th
finish font:341th
finish font:361th
finish font:381th
finish font:401th
time use: 79.19441270828247
(89579520,)
(405, 54, 64, 64)


In [5]:
np.save('fontarray.npy', data)

# data = np.load('fontarray.npy')

In [26]:
with open('fonts_name.txt', 'w') as f:
    for item in fonts:
        f.write(f"{item[:-4]}\n")

# with open('fonts_name.txt', 'r') as f:
#     fonts = f.readlines()
#     fonts = list(map(lambda x:x.strip(), fonts))

In [6]:
for i, font in zip(range(data.shape[0]), fonts):
    plot_font(data[i], font)
    if i % 20 == 0:
        print(f'finish {i}')

finish 0


C:\Users\Thunchakorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


finish 20
finish 40
finish 60
finish 80
finish 100
finish 120
finish 140
finish 160
finish 180
finish 200
finish 220
finish 240
finish 260
finish 280
finish 300
finish 320
finish 340
finish 360
finish 380
finish 400


<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

...

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

In [32]:
plot_all_font_chicken(data, fonts)